In [1]:
# basics
import intake
import xarray as xr
import dask  # memory-efficient parallel computation and delayed execution (lazy evaluation).
import subprocess as sp

In [2]:
%run gem_helpers.ipynb

uid = getpass.getuser()
image_path = make_tempdir("intake_demo_plots")
data_cache_path = make_tempdir("intake_demo_data")

In [3]:
# this function is also part of the gem helpers


def get_list_from_cat(catalog, column):
    """A helper function for getting the contents of a column in an intake catalog.

    Call with the catalog to be inspected and the column of interest."""
    return sorted(catalog.unique(column)[column]["values"])

In [5]:
catalog_file = "/work/ka1081/Catalogs/dyamond-nextgems.json"
cat = intake.open_esm_datastore(catalog_file)
#cat

In [14]:
# a somewhat closer look using get_from_cat
get_from_cat(cat, ["experiment_id", "source_id", "simulation_id",'frequency',"realm"])
# print ("\n".join(get_from_cat (cat, "variable_id")))

,experiment_id,source_id,simulation_id,frequency,realm
0,Cycle1,ICON-SAP-5km,dpp0052,1hr,ocean
1,Cycle1,ICON-SAP-5km,dpp0052,30min,atm
2,Cycle1,ICON-SAP-5km,dpp0052,3hr,atm
3,Cycle1,ICON-SAP-5km,dpp0052,3hr,land
4,Cycle1,ICON-SAP-5km,dpp0052,3hr,ocean
...,...,...,...,...,...
116,nextgems_cycle2,IFS-FESOM,HR0N,6hour,atm
117,nextgems_cycle2,IFS-FESOM,HR2N,1hour,atm
118,nextgems_cycle2,IFS-FESOM,HR2N,6hour,atm
119,nextgems_cycle2,IFS-FESOM,HR2N_nodeep,1hour,atm


In [39]:
#var = "tas"
var = "tas"
#hits = cat.search(simulation_id=["ngc2009", "ngc2012"], variable_id=[var],frequency="1month",realm="atm")

hits = cat.search(simulation_id="hmt0")


hits

,unique
variable_id,96
project,1
institution_id,1
source_id,1
experiment_id,1
simulation_id,1
realm,3
frequency,1
time_reduction,1
grid_label,1


In [41]:
get_from_cat(hits,["grid_label","grid_id","simulation_id", "variable_id"])

,grid_label,grid_id,simulation_id,variable_id
0,gn,unknown,hmt0,"(alb,)"
1,gn,unknown,hmt0,"(albsn,)"
2,gn,unknown,hmt0,"(bldep,)"
3,gn,unknown,hmt0,"(cape,)"
4,gn,unknown,hmt0,"(charnock,)"
...,...,...,...,...
91,gn,unknown,hmt0,"(vis,)"
92,gn,unknown,hmt0,"(vo,)"
93,gn,unknown,hmt0,"(wskin,)"
94,gn,unknown,hmt0,"(wvi,)"


In [31]:
get_from_cat(hits,["grid_label","grid_id","simulation_id"])

,grid_label,grid_id,simulation_id
0,gn,not implemented,ngc2012
1,gr,r360x180,ngc2009


In [16]:
file_cat = {}
#for simulation_id in ("ngc2009", "ngc2012"):
for simulation_id in ("ngc2009","ngc2012"):
    file_cat[simulation_id] = get_list_from_cat(
        hits.search(simulation_id=simulation_id), "uri"
    )

In [26]:
outfile_dict = {}
for simulation_id, files in file_cat.items():
    outfile = f"{data_cache_path}/{var}_timmean_{simulation_id}.nc"
    if True: # not os.access(outfile, os.R_OK):
        query = (
            [
                "cdo",
                "-P",
                "4",
                "-remapcon,r360x180",
                "-timmean",
                "-sellonlatbox,0,360,-30,30",
                "-cat",
                f"-apply,-selname,{var}",
                "[",
            ]
            + files[:10]
            + ["]", outfile]
        )
        # Note, we only use the first 10 files to save time (the [:10] in files[:10]). Remove the [:10] to compute over the whole experiment.
        print(query)
        sp.run(query)
    outfile_dict[simulation_id] = outfile

['cdo', '-P', '4', '-remapcon,r360x180', '-timmean', '-sellonlatbox,0,360,-30,30', '-cat', '-apply,-selname,tas', '[', '/work/bm1235/k203123/quickplots/ngc2009/timmean_monmean_ngc2009_APR_2021-1x1_atm_2d_ml.nc', '/work/bm1235/k203123/quickplots/ngc2009/timmean_monmean_ngc2009_AUG_2021-1x1_atm_2d_ml.nc', '/work/bm1235/k203123/quickplots/ngc2009/timmean_monmean_ngc2009_DEC_2020-1x1_atm_2d_ml.nc', '/work/bm1235/k203123/quickplots/ngc2009/timmean_monmean_ngc2009_DEC_2021-1x1_atm_2d_ml.nc', '/work/bm1235/k203123/quickplots/ngc2009/timmean_monmean_ngc2009_FEB_2021-1x1_atm_2d_ml.nc', '/work/bm1235/k203123/quickplots/ngc2009/timmean_monmean_ngc2009_FEB_2022-1x1_atm_2d_ml.nc', '/work/bm1235/k203123/quickplots/ngc2009/timmean_monmean_ngc2009_JAN_2021-1x1_atm_2d_ml.nc', '/work/bm1235/k203123/quickplots/ngc2009/timmean_monmean_ngc2009_JAN_2022-1x1_atm_2d_ml.nc', '/work/bm1235/k203123/quickplots/ngc2009/timmean_monmean_ngc2009_JUL_2021-1x1_atm_2d_ml.nc', '/work/bm1235/k203123/quickplots/ngc2009/tim

cdo(1) timmean: Process started
cdo(2) sellonlatbox: Process started
cdo(3) cat: Process started
cdo(4) selname: Process started
cdo(5) selname: Process started
cdo(6) selname: Process started
cdo(7) selname: Process started
cdo(8) selname: Process started
cdo(9) selname: Process started
cdo(10) selname: Process started
cdo(11) selname: Process started
cdo(12) selname: Process started
cdo(13) selname: Process started
cdo(4) selname: Processed 64800 values from 35 variables over 1 timestep.
cdo(5) selname: Processed 64800 values from 35 variables over 1 timestep.
cdo(6) selname: Processed 64800 values from 34 variables over 1 timestep.
cdo(7) selname: Processed 64800 values from 35 variables over 1 timestep.
cdo(8) selname: Processed 64800 values from 35 variables over 1 timestep.
cdo(9) selname: Processed 64800 values from 35 variables over 1 timestep.
cdo(10) selname: Processed 64800 values from 34 variables over 1 timestep.
cdo(11) selname: Processed 64800 values from 35 variables ov

['cdo', '-P', '4', '-remapcon,r360x180', '-timmean', '-sellonlatbox,0,360,-30,30', '-cat', '-apply,-selname,tas', '[', '/work/bm1235/k203123/experiments/ngc2012/run_20200201T000000-20200531T235830/ngc2012_atm_2d_1mth_mean_20200301T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2012/run_20200201T000000-20200531T235830/ngc2012_atm_2d_1mth_mean_20200401T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2012/run_20200201T000000-20200531T235830/ngc2012_atm_2d_1mth_mean_20200501T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2012/run_20200201T000000-20200531T235830/ngc2012_atm_2d_1mth_mean_20200601T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2012/run_20200601T000000-20200930T235830/ngc2012_atm_2d_1mth_mean_20200701T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2012/run_20200601T000000-20200930T235830/ngc2012_atm_2d_1mth_mean_20200801T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2012/run_20200601T000000-20200930T235830/ngc2012_atm_2d_1mth_mean_20200901T000000Z.nc', '/work

cdo(4) selname: Processed 5242880 values from 37 variables over 1 timestep.
cdo(5) selname: Processed 5242880 values from 37 variables over 1 timestep.
cdo(6) selname: Processed 5242880 values from 37 variables over 1 timestep.
cdo(7) selname: Processed 5242880 values from 37 variables over 1 timestep.
cdo(8) selname: Processed 5242880 values from 37 variables over 1 timestep.
cdo(9) selname: Processed 5242880 values from 37 variables over 1 timestep.
cdo(10) selname: Processed 5242880 values from 37 variables over 1 timestep.
cdo(11) selname: Processed 5242880 values from 37 variables over 1 timestep.
cdo(12) selname: Processed 5242880 values from 37 variables over 1 timestep.
cdo(13) selname: Processed 5242880 values from 37 variables over 1 timestep.
cdo    remapcon: YAC first order conservative weights from unstructured (2629470) to lonlat (360x180) grid
cdo(3) cat: Processed 52428800 values from 10 variables over 10 timesteps.
cdo(2) sellonlatbox: Processed 52428800 values from 1 